### Packages

In [164]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
sns.set(style="darkgrid")
import sys
sys.path.append('../../src/Vrama')
sys.path.append('../../src')

from Vrama import DataController
from Vrama import Tables
from Vrama import Eye

import time
from scipy import stats


from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show
import pandas as pd
from bokeh.palettes import Spectral4, all_palettes, Set3, Category10

output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

### Methods

In [2]:
def AddIDandSecdf(df):
    cnt = 0
    distractorOnDuration = 0;
    col_id = []
    col_sec = []
    dur_list = []
    dist_list =[]
    current_distractor = ''
    for i,r in df.iterrows():
        if (r['Current Distraction Object'] != current_distractor) & (r['Current Distraction Object'] != '') & (not pd.isnull(r['Current Distraction Object'])):
            cnt +=1
            distractorOnDuration = 0;
        else:
            distractorOnDuration += r.deltaTime
        if pd.isnull(r['Current Distraction Object']):
            current_distractor = np.nan   
            second = np.nan
            dist_id = np.nan
        else:
            current_distractor = r['Current Distraction Object']        
            second = int(distractorOnDuration//1000)
            dist_id = cnt;


        col_id.append(dist_id)
        col_sec.append(second)

    # add back to table 
    df['distraction_id'] = col_id
    df['distraction_sec'] = col_sec
    return df

In [3]:
def BuildSecTable(df):
    id_list = df['distraction_id'].unique()
    table_all = pd.DataFrame()
    for cur_id in id_list:
        cdf = df[df['distraction_id']==cur_id]
        table = GetDistSecTable(cdf)
        table_all = pd.concat([table_all,table]).reset_index(drop=True)
    return table_all

In [4]:
def GetDistSecTable(cdf):
    sec_list = cdf['distraction_sec'].unique()
    distractor_name = cdf['Current Distraction Object'].iloc[0]
    dist_id = cdf['distraction_id'].iloc[0]
    objects_list = []
    total_dur_list = []
    durations_list = []
    sec_dur_list = []
    dist_dur = 0
    for sec in sec_list:

        # filter df by sec
        cdf_sec = cdf[cdf['distraction_sec'] == sec].reset_index(drop=True)

        # calculate total duration
        total_dur_list.append(sum(cdf_sec['deltaTime']))

        #
        current_view_object_duration = 0
        objects = []
        durations = []
        sec_dur = 0

        for i,r in cdf_sec.iterrows():
            dist_dur += r.deltaTime
            sec_dur += r.deltaTime
            if i == 0:
                current_view_object =  r['Active Gaze Object Name']
                current_view_object_duration = 0            
            elif(r['Active Gaze Object Name'] == current_view_object):
                current_view_object_duration += r.deltaTime            
            elif(r['Active Gaze Object Name'] != current_view_object):
                objects.append(current_view_object)
                durations.append(current_view_object_duration)
                current_view_object = r['Active Gaze Object Name']
                current_view_object_duration = 0

            if(i==len(cdf_sec)-1):
                objects.append(current_view_object)
                durations.append(current_view_object_duration)


        objects_list.append(objects)
        durations_list.append(durations)
        sec_dur_list.append(sec_dur)

    dataObjects = {
    'id': dist_id,
    'distractor': distractor_name,
    'sec': sec_list,
    'sec_duration': sec_dur_list,
    'distractor_duration': dist_dur,
    'objects': objects_list,
    'durations': durations_list,
    }

    return pd.DataFrame.from_dict(dataObjects)                

In [5]:
fpath = '../../data/interim/vive/19205/19205_eye_tracking_data_05_24_2019_08_38_35.csv'
df = pd.read_csv(fpath)
df = Eye.PreProcessDf(df)
df = df.reset_index(drop=True)

C:\Users\Greco\Anaconda3\envs\vram\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df = AddIDandSecdf(df)

In [7]:
df = df[~ pd.isnull(df['Current Distraction Object'])].reset_index(drop = True)

In [8]:
# drop no distraction periods
table = BuildSecTable(df)

In [226]:
tableTest = table
for i in list(tableTest.id.unique()):
    dfSample = tableTest[tableTest.id == i].reset_index(drop=True)
    MakeDistractorTimeSeriesPlot(dfSample)

In [227]:
def MakeDistractorTimeSeriesPlot(dfSample):
    #................................................
    objectList = []
    for i,row in dfSample.iterrows():
        for o in row.objects:
            objectList.append(o)
    # extract unique objects
    objectList = list(pd.Series(objectList).unique())
    objectsToRemoveList = ['WhiteBoardWithDisplay','Null']
    for i in objectsToRemoveList:
        if i in objectList:
            objectList.remove(i)
        
    objectDurationList = [[] for i in range(len(objectList))]
    numTimePoints = len(dfSample)
    for i,objectName in enumerate(objectList):
        for ri,row in dfSample.iterrows():
            objectDurationList[i].append(0)
            for oi,o in enumerate(row.objects):
                if o == objectName:
                    objectDurationList[i][ri] += row.durations[oi]
    distractorName = dfSample.iloc[0].distractor
    data = {objectList[i]:objectDurationList[i] for i in range(len(objectList))}
    timePointList = list(range(numTimePoints))
    timePointList = np.array(timePointList) + .5
    data.update([('timepoints',timePointList)])

    #................................................
    distractorName = dfSample.iloc[0].distractor
    data = {objectList[i]:objectDurationList[i] for i in range(len(objectList))}
    timePointList = list(range(numTimePoints))
    timePointList = list(np.array(timePointList) + .5)
    data.update([('timePoints',timePointList)])

    colors = Category10[10] *3
    plotTitle = distractorName + ': Object view duration during distraction'

    if not objectList:
        return
    # figure specs
    p = figure(x_range=[min(timePointList)-.5,max(timePointList)+.5], plot_height=250, plot_width=500,title=plotTitle,y_range=(0, 800),
               toolbar_location=None, tools="")

    p.vbar_stack(objectList, x='timePoints', width=1, color=colors[0:len(objectList)], source=data, legend = [value(x) for x in objectList])
    p.axis.minor_tick_line_color = None
    p.outline_line_color = None
    p.legend.location = "top_right"
    p.legend.orientation = "vertical"
    new_legend = p.legend[0]
    p.legend[0] = None
    p.add_layout(new_legend, 'right')

    p.yaxis.axis_label = "view duration (ms)"
    p.xaxis.axis_label = "Seconds from onset"

    p.legend.label_text_font_size = "8pt"
    p.legend.glyph_height = 10
    p.legend.glyph_width = 10
    show(p)